In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24636
2,Huelva,Confirmados PDIA 14 días,259
3,Huelva,Tasa PDIA 14 días,"50,47060428318101"
4,Huelva,Confirmados PDIA 7 días,118
5,Huelva,Total Confirmados,24856
6,Huelva,Curados,16953
7,Huelva,Fallecidos,305


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24636.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6559.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 5532 personas en los últimos 7 días 

Un positivo PCR cada 2569 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24636.0,118.0,259.0,513170.0,22.994329,50.470604,19.0
Huelva-Costa,15029.0,77.0,155.0,289548.0,26.593173,53.531711,11.0
Condado-Campiña,6895.0,38.0,98.0,156231.0,24.322958,62.727628,7.0
Ayamonte,1200.0,12.0,15.0,21104.0,56.861259,71.076573,4.0
Bollullos Par del Condado,615.0,6.0,13.0,14387.0,41.704316,90.359352,4.0
Cartaya,952.0,12.0,26.0,20083.0,59.752029,129.462730,3.0
Lucena del Puerto,157.0,6.0,11.0,3261.0,183.992640,337.319841,3.0
Puebla de Guzmán,94.0,12.0,14.0,3092.0,388.098318,452.781371,2.0
Almonte,868.0,14.0,32.0,24507.0,57.126535,130.574938,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,94.0,12.0,14.0,3092.0,388.098318,452.781371,2.0
Lucena del Puerto,157.0,6.0,11.0,3261.0,183.992640,337.319841,3.0
Rosal de la Frontera,26.0,1.0,3.0,1697.0,58.927519,176.782557,NaN
Villablanca,176.0,2.0,4.0,2885.0,69.324090,138.648180,0.0
Almonte,868.0,14.0,32.0,24507.0,57.126535,130.574938,1.0
Cartaya,952.0,12.0,26.0,20083.0,59.752029,129.462730,3.0
Palma del Condado (La),975.0,4.0,13.0,10801.0,37.033608,120.359226,-2.0
San Bartolomé de la Torre,195.0,2.0,4.0,3761.0,53.177346,106.354693,0.0
Bollullos Par del Condado,615.0,6.0,13.0,14387.0,41.704316,90.359352,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Moguer,829.0,3.0,12.0,21867.0,13.719303,54.877212,0.0,0.250000
Isla Cristina,1979.0,2.0,7.0,21393.0,9.348852,32.720983,0.0,0.285714
Palma del Condado (La),975.0,4.0,13.0,10801.0,37.033608,120.359226,-2.0,0.307692
Palos de la Frontera,450.0,1.0,3.0,11742.0,8.516437,25.549310,0.0,0.333333
Rosal de la Frontera,26.0,1.0,3.0,1697.0,58.927519,176.782557,0.0,0.333333
Lepe,1994.0,6.0,16.0,27880.0,21.520803,57.388809,0.0,0.375000
Aljaraque,925.0,3.0,8.0,21474.0,13.970383,37.254354,0.0,0.375000
Condado-Campiña,6895.0,38.0,98.0,156231.0,24.322958,62.727628,7.0,0.387755
Almonte,868.0,14.0,32.0,24507.0,57.126535,130.574938,1.0,0.437500
